In [7]:
PROJECT = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your PROJECT
BUCKET = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for AI Platform Training Service
TFVERSION = "1.14"                # TF version for AI Platform Training Service to use

In [8]:
OUTDIR = "gs://{}/taxifare/trained_small".format(BUCKET)

# Exporter 확인

In [9]:
!gsutil ls gs://{BUCKET}/taxifare/trained_small/export/exporter

gs://qwiklabs-gcp-ml-49b827b781ab/taxifare/trained_small/export/exporter/
gs://qwiklabs-gcp-ml-49b827b781ab/taxifare/trained_small/export/exporter/1575556351/


# GCloud 모델에 올리기

In [10]:
VERSION='v1'
!gcloud ai-platform models create taxifare_20191205 --regions us-central1
!gcloud ai-platform versions delete {VERSION} --model taxifare_20191205 --quiet
!gcloud ai-platform versions create {VERSION} --model taxifare_20191205 \
    --origin $(gsutil ls gs://{BUCKET}/taxifare/trained_small/export/exporter | tail -1) \
    --python-version=3.5 \
    --runtime-version {TFVERSION}

Created ml engine model [projects/qwiklabs-gcp-ml-49b827b781ab/models/taxifare_20191205].
ERROR: (gcloud.ai-platform.versions.delete) NOT_FOUND: Field: name Error: The specified model version was not found.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: The specified model version was not found.
    field: name
Creating version (this might take a few minutes)......done.                    


# Prediction

## Invoke prediction REST API via command line

In [11]:
%%writefile ./test.json
{"dayofweek": 1, "hourofday": 0, "pickuplon": -73.885262, "pickuplat": 40.773008, "dropofflon": -73.987232, "dropofflat": 40.732403}

Writing ./test.json


In [13]:
!gcloud ai-platform predict --model=taxifare_20191205 --json-instances=./test.json

PREDICTIONS
[7.731191635131836]


## Invoke prediction REST API via python

In [14]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build("ml", "v1", credentials = credentials,
            discoveryServiceUrl = "https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json")

request_data = {"instances":
  [
      {
        "dayofweek": 1,
        "hourofday": 8,
        "pickuplon": -73.885,
        "pickuplat": 40.773,
        "dropofflon": -73.987,
        "dropofflat": 40.732,
      }
  ]
}

parent = "projects/{}/models/taxifare_20191205".format(PROJECT) # use default version
#parent = "projects/{}/models/taxifare/versions/{}".format(PROJECT,VERSION) # specify a specific version

response = api.projects().predict(body = request_data, name = parent).execute()
print("response = {0}".format(response))

response = {'predictions': [{'predictions': [9.171065330505371]}]}
